In [1]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid import config
from sem_covid.services.store_registry import store_registry
import pandas as pd

In [2]:
DS_UNIFIED_TEST_DATASET_FILE_NAME = 'ds_unified_test_dataset.json'
MINIO_BUCKET = 'rdf-transformer'
MINIMIZE_COLUMNS_DATA = ['title','content', 'content_cleaned_topic_modeling',
                         'document_embeddings_use', 'document_embeddings_eurlex_bert', 'topic_embeddings_eurlex_bert']

In [3]:
df = store_registry.es_index_store().get_dataframe(index_name='ds_unified_dataset')
test_df = pd.DataFrame(df.head(100))

100% (6360 of 6360) |####################| Elapsed Time: 0:00:15 Time:  0:00:15


In [4]:
test_df.columns

Index(['title', 'content', 'content_cleaned_topic_modeling', 'date',
       'doc_source', 'country', 'pwdb_category', 'pwdb_target_group_l1',
       'pwdb_funding', 'pwdb_type_of_measure', 'pwdb_actors',
       'document_embeddings_use', 'document_embeddings_eurlex_bert',
       'topic_embeddings_eurlex_bert', 'eu_cellar_subject_matter_labels',
       'eu_cellar_resource_type_labels', 'eu_cellar_directory_code_labels',
       'eu_cellar_author_labels', 'pwdb_target_group_l2', 'ireland_keyword',
       'ireland_department_data', 'ireland_campaign', 'ireland_page_type',
       'eu_timeline_topic'],
      dtype='object')

In [5]:
for column in MINIMIZE_COLUMNS_DATA:
    test_df[column] = test_df[column].apply(lambda x: x[:5])
test_df.reset_index(drop=True,inplace=True)

In [6]:
test_df['index'] = test_df.index

In [7]:
minio = store_registry.minio_object_store(minio_bucket=MINIO_BUCKET)
minio.put_object(object_name=DS_UNIFIED_TEST_DATASET_FILE_NAME,content=test_df.to_json(orient='index'))

109306

In [8]:
tmp_df = pd.read_json(minio.get_object(object_name=DS_UNIFIED_TEST_DATASET_FILE_NAME), orient='index')

In [9]:
tmp_df.head(5)

,title,content,content_cleaned_topic_modeling,date,doc_source,country,pwdb_category,pwdb_target_group_l1,pwdb_funding,pwdb_type_of_measure,...,eu_cellar_resource_type_labels,eu_cellar_directory_code_labels,eu_cellar_author_labels,pwdb_target_group_l2,ireland_keyword,ireland_department_data,ireland_campaign,ireland_page_type,eu_timeline_topic,index
0,COMMI,COMMI,commi,2021-01-12,ds_eu_cellar,European Union,"Protection of workers, adaptation of workplace",[businesses],"[European Funds, National funds, Regional funds]",Other initiatives or policies,...,[Staff working document],[],[Directorate-General for Migration and Home Af...,[],[],[],[],[],[],0
1,Regul,Regul,regul,2021-02-16,ds_eu_cellar,European Union,Ensuring business continuity and support for e...,[workers],[No special funding required],Legislations or other statutory regulations,...,[Regulation],[],"[Council of the European Union, European Parli...",[],[],[],[],[],[],1
2,COMMU,COMMU,commu,2021-02-03,ds_eu_cellar,European Union,Reorientation of business activities,[],"[European Funds, Regional funds]",Company practices,...,[Communication],[],[Directorate-General for Health and Food Safet...,[],[],[],[],[],[],2
3,Europ,Europ,europ,2020-05-15,ds_eu_cellar,European Union,"Promoting the economic, labour market and soci...",[],"[European Funds, National funds, Regional funds]",Legislations or other statutory regulations,...,[Resolution],[],[European Parliament],[],[],[],[],[],[],3
4,REPOR,REPOR,repor,2021-09-09,ds_eu_cellar,European Union,"Protection of workers, adaptation of workplace",[workers],[No special funding required],Legislations or other statutory regulations,...,[Report],[],"[European Commission, Secretariat-General]",[],[],[],[],[],[],4
